# Feature Extraction and t-SNE Visualization

In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm
from dataprocessor.datacomposer import getData
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
import clip
import os
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from sklearn.manifold import TSNE
import hyperparameters as HP

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
print("CUDA_VISIBLE_DEVICES: ", os.environ["CUDA_VISIBLE_DEVICES"])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Load the CLIP model
model, _ = clip.load(HP.clip_model, device=device)

# Define transforms for image processing
transform = Compose([
    ToTensor(),
    Resize(256),
    CenterCrop(224),
    Normalize((0.48145466, 0.4578275, 0.40821073),
              (0.26862954, 0.26130258, 0.27577711)),
])

# Load CIFAR10 test dataset
cifar_test_dataset = CIFAR10(
    root='data/', train=False, download=True, transform=transform)

In [ ]:
# Extract features using CLIP model
X = []
y = []
for i in tqdm(range(len(cifar_test_dataset))):
    img, label = cifar_test_dataset[i]
    img = img.unsqueeze(0).to(device)
    with torch.no_grad():
        image_features = model.encode_image(img)
        image_features /= image_features.norm(dim=-1, keepdim=True)
    X.append(image_features.cpu().numpy())
    y.append(label)
X = np.concatenate(X, axis=0)
y = np.array(y)

# Perform t-SNE dimensionality reduction
X_embedded = TSNE(n_components=2, learning_rate=100).fit_transform(X)

CIFAR10 class information

0: airplane, 1: automobile, 2: bird, 3: cat, 4: deer
5: dog, 6: frog, 7: horse, 8: ship, 9: truck

Define color map for visualization

Group classes into two categories:

Artificial objects (0, 1, 8, 9) and animals (2, 3, 4, 5, 6, 7)

In [ ]:
cmap = {0: '#03045e', 1: '#023e8a', 8: '#0077b6', 9: '#00b4d8',
        2: '#7a0103', 3: '#8e0103', 4: '#a50104', 5: '#b81702', 6: '#eb1d1d', 7: '#f72634'}

# Visualize the t-SNE results
plt.figure(figsize=(10, 10))
for i in range(10):
    indices = y == i
    plt.scatter(X_embedded[indices, 0],
                X_embedded[indices, 1], c=cmap[i], label=i, alpha=0.6)
plt.legend()
plt.savefig('tsne_visualization_{}.png'.format(HP.clip_model))
plt.show()

use SCLRE to visualize the features

In [ ]:
import hyperparameters as HP
from model.TC2 import TransformerContrastive
from model.TC2 import get_backbone
HP.cls_num = 2
model_path = ''
checkpoint = torch.load(model_path)  # Path to the checkpoint file
model = TransformerContrastive()
model.load_state_dict(checkpoint['model'])

In [ ]:
test_loader = DataLoader(cifar_test_dataset, batch_size=64, shuffle=False)
model = model.to(device)
model.eval()

X = []
y = []

with torch.no_grad():
    for images, labels in tqdm(test_loader):
        images = images.to(device)
        image_features, _ = model(images)

        image_features = image_features / \
            image_features.norm(dim=-1, keepdim=True)

        X.append(image_features.cpu().numpy())
        y.append(labels.numpy())
X = np.concatenate(X, axis=0)
y = np.concatenate(y, axis=0)

In [ ]:
tsne = TSNE(n_components=2, learning_rate=100, random_state=42)
X_embedded = tsne.fit_transform(X)
cmap = {0: '#03045e', 1: '#023e8a', 8: '#0077b6', 9: '#00b4d8',
        2: '#7a0103', 3: '#8e0103', 4: '#a50104', 5: '#b81702', 6: '#eb1d1d', 7: '#f72634'}
plt.figure(figsize=(10, 10))
for i in range(X_embedded.shape[0]):
    plt.scatter(X_embedded[i, 0], X_embedded[i, 1],
                c=cmap[y[i]], edgecolors='none', s=10, alpha=0.7)
plt.axis('off')
plt.savefig('tsne_on_{}.pdf'.format(model_path), dpi=500, bbox_inches='tight')
plt.show()